1. Open the page in a separate Chrome/Firefox tab and use something like an
Inspect Element tool to view the source HTML and understand its structure

2. Read the page using bs4

In [7]:
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
wiki_page = "/Users/theranmeadows/Desktop/dsc540datapreparation/week5&6/data/List of countries by GDP (nominal) - Wikipedia.html"

with open( wiki_page, "r") as fd:
    soup = BeautifulSoup(fd)


3. Find the table structure you will need to deal with (how many tables there are?)

In [24]:
tables = soup.find_all("table")
len(tables)

7

7 tables

4. Find the right table using bs4

In [40]:
data_table = soup.find("table", {"class": '"wikitable"|}'})
print(type(data_table))

<class 'NoneType'>


5. Separate the source names and their corresponding data

In [35]:
sources = data_table.tbody.findAll('tr', recursive=False)[0]
sources_list = [td for td in sources.findAll('td')]
print(len(sources_list))

AttributeError: 'NoneType' object has no attribute 'tbody'